### Опишем цели исследования:

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Моя первая задача как аналитика — проанализировать базу данных.

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных
#### Таблица books:
Содержит данные о книгах:
- book_id — идентификатор книги;
- author_id — идентификатор автора;
- title — название книги;
- num_pages — количество страниц;
- publication_date — дата публикации книги;
- publisher_id — идентификатор издателя.

#### Таблица authors:
Содержит данные об авторах:
- author_id — идентификатор автора;
- author — имя автора.

#### Таблица publishers:
Содержит данные об издательствах:
- publisher_id — идентификатор издательства;
- publisher — название издательства.

#### Таблица ratings:
Содержит данные о пользовательских оценках книг:
- rating_id — идентификатор оценки;
- book_id — идентификатор книги;
- username — имя пользователя, оставившего оценку;
- rating — оценка книги.

#### Таблица reviews:
Содержит данные о пользовательских обзорах:
- review_id — идентификатор обзора;
- book_id — идентификатор книги;
- username — имя автора обзора;
- text — текст обзора.

### Подключение к базе данных

In [1]:
# !pip install sqlalchemy
!pip install --force-reinstall 'sqlalchemy<2.0.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.20
    Uninstalling SQLAlchemy-2.0.20:
      Successfully uninstalled SQLAlchemy-2.0.20
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


<div class="alert alert-warning"><b>Комментарий ревьюера</b>&nbsp;🔧<br>
Я заметил что первая ячейка имела номер выполнения, отличный от <tt>[1]</tt>. Это может приводить к тому, что не вызовут ошибку какие-то изменения кода, которые вызвали бы ошибку после перезапуска ноутбука (или при его запуске на другой машине), например потому, что в памяти сохраняются переменные, которые были сформированы до того, как код был изменён. Поэтому важно выполнять ячейки с первой по последнюю строго последовательным образом перед тем, как отправлять их кому-то.
</div>

<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Прощу прощения, перед отправкой буду перезапускать в обязательном порядке<br>
</div>

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import display, HTML

In [3]:
# устанавливаем параметры
db_config = {'user': 'leonidas', # имя пользователя
             'pwd': 'HzMQksJE3m', # пароль
             'host': '89.108.115.89',
             'port': 13482, # порт подключения
             'db': 'yogadata'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],db_config['pwd'],db_config['host'],db_config['port'],db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;📝<br>
Это код из справки, но его можно улучшить. Если задать имена ключам в текстовом шаблоне, то можно гораздо короче и удобнее формировать строки:

    'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
</div>

<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Спасибо! Пометил себе.
</div>

In [4]:
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,clients,leonidas,None,False,False,False,False
1,public,subscriptions,leonidas,None,True,False,False,False
2,public,visits,leonidas,None,True,False,False,False


In [9]:
# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT
    table_name,
    column_name,
    data_type,
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name IN ('clients', 'subscriptions', 'visits')
ORDER BY table_name, column_name;
''', con = engine))

,table_name,column_name,data_type,is_nullable
0,clients,gender,text,YES
1,clients,unid,text,YES
2,subscriptions,abonement,text,YES
3,subscriptions,client_id,text,YES
4,subscriptions,ctime,timestamp without time zone,NO
5,subscriptions,end_date,date,YES
6,subscriptions,lessons_left,integer,YES
7,subscriptions,mtime,timestamp without time zone,NO
8,subscriptions,sdate,timestamp without time zone,NO
9,subscriptions,start_date,date,YES


### Исследуем таблицы — выведем первые строки:

In [45]:
# смотрим на значения в 5 первых строках
display(pd.io.sql.read_sql('''
SELECT *
FROM clients
ORDER BY unid
LIMIT 5;
''', con = engine))

,unid,gender
0,0046ebbaf63abc7150d07d1ae,F
1,0073e6829cbc67dc1e1467a07,M
2,00a7fa1e460cd64eae45d6b8f,F
3,0124a52e4e6b6e365402b66d7,F
4,0138e8b4dd238aa77c324e614,M


In [18]:
# Посмотрим есть ли значения null:
display(pd.io.sql.read_sql('''
SELECT COUNT(*) AS row
FROM clients
WHERE (gender IS NULL) OR (unid IS NULL)
LIMIT 5;
''', con = engine))

,row,unique_unid
0,0,0


In [20]:
# Посмотрим есть ли дубли в unid
display(pd.io.sql.read_sql('''
SELECT COUNT(*) AS row, COUNT(DISTINCT unid) AS unique_unid
FROM clients
LIMIT 5;
''', con = engine))

,row,unique_unid
0,1395,1395


In [21]:
# Посмотрим есть ли дубли в unid
display(pd.io.sql.read_sql('''
SELECT gender, COUNT(unid) AS count_unid
FROM clients
GROUP BY gender
LIMIT 5;
''', con = engine))

,gender,count_unid
0,M,60
1,U,94
2,F,1241


Мужчин в 20 раз меньше

In [32]:
# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT *
FROM subscriptions
LIMIT 5;
''', con = engine))

,sdate,abonement,client_id,lessons_left,start_date,end_date,ctime,mtime
0,2023-09-11 22:25:05,5 занятий,7989580efbcd02e1226addbd9,4,2023-09-11,2023-11-11,2023-09-12 20:35:10.891991,2023-09-12 20:49:25.133746
1,2023-09-08 11:26:17,5 занятий,7fc6446ef711b721fbb35fb63,1,2023-09-08,2023-11-08,2023-09-12 20:35:10.891991,2023-09-14 18:47:48.976594
2,2023-08-18 00:47:51,5 занятий,c53a3bfcd09fb9d493ca9d511,1,2023-09-02,2023-11-02,2023-09-12 20:30:07.494995,2023-09-14 18:47:48.976594
3,2023-08-01 13:04:49,20 занятий,db4af14a318d39c767e19877b,17,2023-07-31,2024-02-01,2023-09-12 20:30:07.494995,2023-09-14 18:47:48.976594
4,2023-07-25 19:03:44,5 занятий,47b0ebaec83e50887c1070e32,2,2023-07-24,2023-09-23,2023-09-12 20:30:07.494995,2023-09-14 18:47:48.976594


?_что за столбцы ctime 	mtime? (у Дмитрий спросил)

In [24]:
# Посмотрим есть ли значения null:
display(pd.io.sql.read_sql('''
SELECT COUNT(*) FILTER (WHERE sdate IS NULL) AS row_sdate_null
  , COUNT(*) FILTER (WHERE abonement IS NULL) AS row_abonement_null
  , COUNT(*) FILTER (WHERE client_id IS NULL) AS row_client_id_null
  , COUNT(*) FILTER (WHERE lessons_left IS NULL) AS row_lessons_left_null
  , COUNT(*) FILTER (WHERE start_date IS NULL) AS row_start_date_null
  , COUNT(*) FILTER (WHERE end_date IS NULL) AS row_end_date_null
  , COUNT(*) FILTER (WHERE ctime IS NULL) AS row_ctime_null
  , COUNT(*) FILTER (WHERE mtime IS NULL) AS row_mtime_null
FROM subscriptions
LIMIT 5;
''', con = engine))

,row_sdate_null,row_abonement_null,row_client_id_null,row_lessons_left_null,row_start_date_null,row_end_date_null,row_ctime_null,row_mtime_null
0,0,2,0,0,0,0,0,0


!_Есть пропущенные столбцы

In [27]:
# Посмотрим сколько всего абонементов
display(pd.io.sql.read_sql('''
SELECT COUNT(*) AS count_abonement
FROM subscriptions
LIMIT 5;
''', con = engine))

,count_abonement
0,785


In [30]:
# Посмотрим сколько клиентов покупают абонементы
display(pd.io.sql.read_sql('''
SELECT COUNT(DISTINCT client_id) AS unique_client_id
FROM subscriptions
LIMIT 5;
''', con = engine))

,unique_client_id
0,363


In [43]:
# Посмотрим client_id - в таблице с записями есть типа nda (по итогу просмотра тут таких нет)
display(pd.io.sql.read_sql('''
SELECT DISTINCT client_id
FROM subscriptions
ORDER BY client_id
LIMIT 5;
''', con = engine))

,client_id
0,01a8461e27ceb60e60c8bd52a
1,0269d3117391d8e9ffd14277d
2,02c09ed3b957487d11cfc319e
3,02d848efe2926e48d44cc359c
4,02ee5d0a76790fc795e3f498e


In [29]:
# Посмотрим сколько каких абонементов
display(pd.io.sql.read_sql('''
SELECT abonement, COUNT(client_id) AS count_client_id
FROM subscriptions
GROUP BY abonement
ORDER BY abonement;
''', con = engine))

,abonement,count_client_id
0,10 занятий,152
1,20 занятий,42
2,5 занятий,386
3,8 занятий (детский),38
4,Бесплатное занятие,30
5,Весна-2020,3
6,Подарочный сертификат,3
7,Пригласительный,37
8,Пробный безлимитный,92
9,None,2


In [31]:
# Посмотрим диапазоны дат и мин и макс оставшихся занятий
display(pd.io.sql.read_sql('''
SELECT MIN(sdate) AS min_sdate, MAX(sdate) AS max_sdate
  , MIN(start_date) AS min_start_date, MAX(start_date) AS max_start_date
  , MIN(end_date) AS min_end_date, MAX(end_date) AS max_end_date
  , MIN(ctime) AS min_ctime, MAX(ctime) AS max_ctime
  , MIN(mtime) AS min_mtime, MAX(mtime) AS max_mtime
  , MIN(lessons_left) AS min_lessons_left, MAX(lessons_left) AS max_lessons_left
FROM subscriptions
LIMIT 5;
''', con = engine))

,min_sdate,max_sdate,min_start_date,max_start_date,min_end_date,max_end_date,min_ctime,max_ctime,min_mtime,max_mtime,min_lessons_left,max_lessons_left
0,2020-03-06 13:19:46,2023-09-11 22:26:50,2019-10-14,2023-09-18,2020-03-08,2024-02-01,2023-09-12 20:30:07.494995,2023-09-12 20:35:10.891991,2023-09-12 20:35:10.891991,2023-09-14 18:47:48.976594,-1,17


In [41]:
# смотрим есть ли клиенты с абонементами, которых нет в клиентах
display(pd.io.sql.read_sql('''
SELECT COUNT(*)
FROM subscriptions
WHERE client_id NOT IN (SELECT unid FROM clients)
LIMIT 5;
''', con = engine))

,count
0,0


In [13]:
# смотрим на ___
display(pd.io.sql.read_sql('''
SELECT *
FROM visits
LIMIT 5;
''', con = engine))

,aid,lesson_id,client_id,status,book_date,visit_date,trainer_id,lesson,kids,ctime,mtime
0,24983,24979,51446b89f58e6e73d3b421c8d,booked,2023-09-09 00:28:09,2023-09-19,2e3e9e5e62f53bcfdd4ef49f7,Йога в гамаках. Дети 12-15 лет. ОТКРЫТЫЙ УРОК,False,2023-09-12 22:51:23.673085,2023-09-12 22:51:23.673085
1,24988,24921,579d883c9fb3055e3c981786c,booked,2023-09-10 17:42:44,2023-09-18,e93a228046b0e849b613db6c6,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-12 22:51:23.673085
2,24935,24855,579d883c9fb3055e3c981786c,booked,2023-09-06 17:02:43,2023-09-17,24afa988163406000dd31a78d,АЭРОЙОГА + ЙОГА-НИДРА,False,2023-09-12 22:51:23.673085,2023-09-12 22:51:23.673085
3,24986,24985,c53a3bfcd09fb9d493ca9d511,booked,2023-09-10 13:08:59,2023-09-17,0ed1eb34ce87d36b41b5dac1b,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-12 22:51:23.673085
4,24984,24847,c53a3bfcd09fb9d493ca9d511,booked,2023-09-10 13:01:24,2023-09-14,bc18610c215b8fc4042cce7b4,Хатха - йога,False,2023-09-12 22:51:23.673085,2023-09-12 22:51:23.673085


In [35]:
# смотрим есть ли клиенты с записями, которых нет в клиентах
display(pd.io.sql.read_sql('''
SELECT COUNT(*)
FROM visits
WHERE client_id NOT IN (SELECT unid FROM clients)
LIMIT 15;
''', con = engine))

,count
0,11


In [37]:
# смотрим есть ли клиенты с записями, которых нет в клиентах
display(pd.io.sql.read_sql('''
SELECT *
FROM visits
WHERE client_id NOT IN (SELECT unid FROM clients)
LIMIT 15;
''', con = engine))

,aid,lesson_id,client_id,status,book_date,visit_date,trainer_id,lesson,kids,ctime,mtime
0,24746,24585,"nda_(3,8)",payed,2023-08-24 10:21:50,2023-08-24,d8e976c9b9a9f9a4609a4413c,Хатха - йога,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
1,22961,22679,"nda_(24,32)",payed,2023-03-28 09:41:15,2023-03-30,24afa988163406000dd31a78d,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
2,20687,20386,"nda_(49,9)",payed,2022-12-05 17:49:39,2022-12-05,9b4321a6001006da1a1e56d11,Арома - Аэройога,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
3,16674,16501,"nda_(100,33)",payed,2022-02-11 10:05:51,2022-02-15,e93a228046b0e849b613db6c6,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
4,16152,16028,"nda_(105,36)",payed,2022-01-11 14:32:44,2022-01-15,a10321740f26a1da16645e644,Аэростретчинг с Александрой,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
5,14059,13834,"nda_(137,8)",payed,2021-08-28 22:17:40,2021-08-29,a10321740f26a1da16645e644,АЭРОСТРЕТЧИНГ (растяжка с гамаком),False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
6,9978,9712,"nda_(193,10)",payed,2021-01-27 20:13:38,2021-01-30,0ed1eb34ce87d36b41b5dac1b,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
7,7236,7117,"nda_(228,2)",canceled,2020-09-05 19:38:27,2020-09-06,None,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
8,7243,7073,"nda_(228,9)",payed,2020-09-05 19:49:47,2020-09-05,9b4321a6001006da1a1e56d11,Арома - Аэройога,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322
9,3568,3417,"nda_(234,2)",canceled,2020-07-23 16:36:23,2020-07-23,0ed1eb34ce87d36b41b5dac1b,АЭРОЙОГА,False,2023-09-12 22:51:23.673085,2023-09-14 18:50:11.929322


подобных значений в client_id (типа nda) на момент 20230915 не увидел

In [38]:
# смотрим есть ли клиенты с записями, которых нет в клиентах
display(pd.io.sql.read_sql('''
SELECT *
FROM clitnts
WHERE client_id NOT IN (SELECT unid FROM clients)
LIMIT 15;
''', con = engine))

NameError: ignored

In [ ]:
# выведем первые пять строк таблицы books - она содержит данные о книгах:
query = '''
SELECT *
FROM books
LIMIT 5 '''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;<br>
✔️ Хорошо, что ограничиваешь число загружаемых в память локальной машины строк средствами SQL, для крупных таблиц это будет критически важно

Круто что ты ниже отдельным запросом смотришь на другие статистики
</div>

In [ ]:
# выведем наибольшее и наименьшее кол-во страниц и самую раннюю и самую позднюю даты публикации, кол-во строк:
query = '''
SELECT  MAX(num_pages) AS max_num_pages,
        MIN(num_pages) AS min_num_pages,
        MAX(publication_date) AS max_date,
        MIN(publication_date) AS min_date,
        COUNT(*)
FROM books
'''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,max_num_pages,min_num_pages,max_date,min_date,count
0,2690,14,2020-03-31,1952-12-01,1000


In [ ]:
# далее выведем пять первых строк таблицы authors - она содержит данные об авторах:
query = '''
SELECT *
FROM authors
LIMIT 5 '''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [ ]:
# далее выведем пять первых строк таблицы publishers - она содержит данные об издательствах:
query = '''
SELECT *
FROM publishers
LIMIT 5 '''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [ ]:
# далее выведем пять первых строк таблицы ratings - она содержит данные о пользовательских оценках книг:
query = '''
SELECT *
FROM ratings
LIMIT 5 '''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [ ]:
# посмотрим максимальную и минимальную оценку книг:
query = '''
SELECT MAX(rating) AS max_rating,
       MIN(rating) AS min_rating
FROM ratings
'''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,max_rating,min_rating
0,5,1


In [ ]:
# на последок выведем пять первых строк таблицы reviews - она содержит данные о пользовательских обзорах:
query = '''
SELECT *
FROM reviews
LIMIT 5 '''

In [ ]:
engine.execute(query);

In [ ]:
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;📝<br>
Хорошо! Мы познакомились с данными: посмотрели содержимое таблиц и заодно проверили соединение с сервером. Ниже некоторые полезные запросы, которые могут помочь в будущем
</div>

In [ ]:
# КОД РЕВЬЮЕРА
# какие ещё таблицы доступны в базе
display(pd.io.sql.read_sql('''

SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND
      schemaname != 'information_schema';

''', con = engine))


# смотрим на типы столбцов в интересующих таблицах
display(pd.io.sql.read_sql('''
SELECT
    table_name,
    column_name,
    data_type,
    is_nullable
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name IN ('books', 'authors', 'publishers', 'ratings', 'reviews');
''', con = engine))

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,orders,praktikum_admin,None,True,False,False,False
1,public,visits,praktikum_admin,None,True,False,False,False
2,public,advertisment_costs,praktikum_admin,None,True,False,False,False
3,public,authors,praktikum_admin,None,True,False,True,False
4,public,second,praktikum_student,None,False,False,False,False
5,public,second_b,praktikum_student,None,False,False,False,False
6,public,publishers,praktikum_admin,None,True,False,True,False
7,public,author,praktikum_student,None,True,False,False,False
8,public,reviews,praktikum_admin,None,True,False,True,False
9,public,ratings,praktikum_admin,None,True,False,True,False


,table_name,column_name,data_type,is_nullable
0,authors,author_id,integer,NO
1,authors,author,text,YES
2,publishers,publisher_id,integer,NO
3,publishers,publisher,text,YES
4,reviews,review_id,integer,NO
5,reviews,book_id,integer,YES
6,reviews,username,text,YES
7,reviews,text,text,YES
8,ratings,rating_id,integer,NO
9,ratings,book_id,integer,YES


<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Опять спасибо! Каждый твой совет делает жизнь аналитика понятнее)
</div>

### Посчитаем, сколько книг вышло после 1 января 2000 года:

In [ ]:
# Сделаем одним SQL-запрос для решения задания, выведем результат запроса в тетрадке:

In [ ]:
display(pd.io.sql.read_sql('''
SELECT COUNT(*)
FROM books
WHERE publication_date > '2000-01-01'
''', con = engine))

,count
0,819


In [ ]:
display(pd.io.sql.read_sql('''
SELECT COUNT(*)
FROM books
WHERE publication_date >= '2000-01-01'
''', con = engine))

,count
0,821


#### Опишем выводы:
С начала 2000 года вышло 821 книга из присутствующих БД
После 01 января 2000 года вышло 819 книг из присутствующих БД

<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;<br>
✔️ Тут всё верно. Сработал бы и запрос с таким условием:

    publication_date >= '2000-01-01'

Если следовать вопросу буквально, то корректно делать строгое сравнение, не включая книги, опубликованные 1 января 2000 года, но по опыту обычно нужны данные с самого начала года. В подобных ситуациях стоит уточнять что именно нужно или сделать оба варианта, если это невозможно.
    
💡 Желательно сопровождать запрос комментариями (их можно начинать с <tt>--</tt>) о том, что в нём происходит, это хорошая привычка, она очень облегчает чтение запросов, особенно длинных.
</div>

<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Сделал оба варианта.
</div>

### Для каждой книги посчитайте количество обзоров и среднюю оценку:

In [ ]:
# Тут у тимлида надо спросить: "Какие характеристики книг выводить и сколько книг?"
# Сделаем одним SQL-запрос для решения задания, выведем результат запроса в тетрадке:

In [ ]:
display(pd.io.sql.read_sql('''
WITH

-- посчитаем количество обзоров для каждой книги
count_texts AS (SELECT book_id, COUNT(text) AS count_text
                FROM reviews
                GROUP BY book_id
                ),

-- посчитаем среднюю оценку для каждой книги
avg_ratings AS (SELECT book_id, ROUND(AVG(rating), 2) AS avg_rating
                FROM ratings
                GROUP BY book_id
                )

SELECT b.book_id, author_id, title, num_pages, publication_date, publisher_id, count_text, avg_rating
FROM books AS b
LEFT OUTER JOIN count_texts AS ct ON ct.book_id = b.book_id
LEFT OUTER JOIN avg_ratings AS ar ON ar.book_id = b.book_id
ORDER BY avg_rating DESC -- отсортируем по средней оценке

''', con = engine))

,book_id,author_id,title,num_pages,publication_date,publisher_id,count_text,avg_rating
0,17,94,A Dirty Job (Grim Reaper #1),387,2006-03-21,331,4.0,5.00
1,610,250,Tai-Pan (Asian Saga #2),734,2009-09-01,80,2.0,5.00
2,553,257,School's Out—Forever (Maximum Ride #2),409,2006-05-23,176,3.0,5.00
3,513,178,Piercing the Darkness (Darkness #2),448,2003-06-26,67,2.0,5.00
4,902,558,The War of Art: Break Through the Blocks & Win...,168,2003-04-01,326,2.0,5.00
...,...,...,...,...,...,...,...,...
995,915,580,The World Is Flat: A Brief History of the Twen...,616,2006-04-18,106,3.0,2.25
996,316,321,His Excellency: George Washington,320,2005-11-08,309,2.0,2.00
997,371,625,Junky,208,2003-04-01,210,2.0,2.00
998,202,91,Drowning Ruth,368,2001-07-31,33,3.0,2.00


#### Опишем выводы:
Вывели каждой книги количество обзоров и среднюю оценку, упорядочив по средней оценке

<div class="alert alert-warning"><b>Комментарий ревьюера</b>&nbsp;🔧<br>
Ты обратил внимание на количество выведенных записей и на общее количество книг. Выведено меньше книг, чем есть в базе, а нам нужно вывести все книги. Внимательнее посмотри на то, как ты объединяешь таблицы.
</div><div class="alert alert-warning">💡 Лучше сортировать подобные таблицы по каким-то показателям.</div>

<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Изменил объединение, спасибо.
</div>

<div class="alert alert-success"><b>Комментарий ревьюера 2&nbsp;<big>👍</big></b></div>

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры:

In [ ]:
# Сделаем однин SQL-запрос для решения задания, выведем результат запроса в тетрадке:

In [ ]:
display(pd.io.sql.read_sql('''
SELECT p.publisher, COUNT (book_id) AS count_books
FROM books AS b
LEFT OUTER JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50 -- книги в которых больше 50 страниц
GROUP BY p.publisher -- сгруппируем по издателю
ORDER BY count_books DESC -- упорядочим по убыванию по кол-ву книг
LIMIT 1 -- возьмем издателя у которого больше всего выпущено книг
''', con = engine))

,publisher,count_books
0,Penguin Books,42


#### Опишем выводы:
Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, кол-во выпущенных этим издательством книг попадающих под заданный критерий составляет 42.

<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;👍<br>
Совершенно верно, это правильное издательство и оно одно, как нас просили.
</div>

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками:

In [ ]:
# Сделаем однин SQL-запрос для решения задания, выведем результат запроса в тетрадке:

In [ ]:
display(pd.io.sql.read_sql('''
WITH

-- отобрали книги у которых 50 и более оценок и посчитали среднюю оценку таких книг
avg_ratings AS (SELECT MIN(b.author_id) AS author_id,  AVG(rating) AS avg_rating
                FROM ratings AS r
                LEFT OUTER JOIN books AS b ON r.book_id = b.book_id
                GROUP BY b.book_id
                HAVING COUNT(rating_id) >= 50 -- взяли книги у которых более 50 оценок
                )

SELECT author, ROUND(AVG(avg_rating),2) AS avg_avg_rating
FROM avg_ratings AS ar
LEFT OUTER JOIN authors AS a ON a.author_id = ar.author_id  -- ко всем author_id определенным во врем.таблице добавим имя автора
GROUP BY a.author_id  -- сгруппируем по автору
ORDER BY avg_avg_rating DESC --упорядочим по среднему средней оценки по убыванию
LIMIT 1 -- возьмем автора с наивысшей оценкой


''', con = engine))

,author,avg_avg_rating
0,J.K. Rowling/Mary GrandPré,4.28


<div class="alert alert-danger"><b>Комментарий ревьюера</b>&nbsp;🔧<br>
Не выведен сам рейтинг и нету группировки по авторам - по сути выведен автор самой рейтинговой книги из тех, что с 50 страницами и более</div>

<div class="alert alert-info"> <b>Комментарии студента:</b><br>
    Исправил, не так прочитал и понял задание. Спасибо.
</div>

<div class="alert alert-success"><b>Комментарий ревьюера 2&nbsp;<big>👍</big></b></div>

#### Опишем выводы:
Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré, среднее средних оценок его книг (с 50 и более оценками) - 4.28.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [ ]:
# Сделаем однин SQL-запрос для решения задания, выведем результат запроса в тетрадке:

In [ ]:
display(pd.io.sql.read_sql('''
SELECT ROUND(AVG(count_text), 2)
FROM (SELECT COUNT(text) AS count_text
      FROM reviews
      WHERE username IN (SELECT username
                         FROM ratings
                         GROUP BY username
                         HAVING COUNT(rating) > 50
                        )
      GROUP BY username
     ) AS count_texts
''', con = engine))

,round
0,24.33


#### Опишем выводы:
Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24,33

<div class="alert alert-success"><b>Комментарий ревьюера</b>&nbsp;✔️<br>
И это... правильный ответ!
</div>

<div class="alert alert-warning"><b>Комментарий ревьюера</b>&nbsp;💡<br>
Сам себя не похвалишь, не подчеркнёшь важность и сложность своей работы — никто этого делать за тебя не будет. Выводы — как раз такое место, где можно напомнить тому, кто будет смотреть отчёт, что тут не тяп-ляп и в продакшн, а была проведена серьёзная работа, глубокий анализ, <s>зарплату аналитику хорошо бы поднять</s>, кратко напомнить о том, что было достигнуто, читатель может быть к концу уже об этом забыл.
</div>


### Выводы:

В ходе выполнения проекта мы познакомились с данными: посмотрели содержимое таблиц и заодно проверили соединение с сервером.

Анализируя содержимое предоставленной БД получили следующую информацию:
- С начала 2000 года вышло 821 книга из присутствующих БД, после 01 января 2000 года вышло 819 книг из присутствующих БД;
- Вывели каждой книги количество обзоров и среднюю оценку, упорядочив по средней оценке;
- Наибольшее число книг толще 50 страниц выпустило издательство Penguin Books, кол-во выпущенных этим издательством книг попадающих под заданный критерий составляет 42;
- Автор с самой высокой средней оценкой книг (учитывая только книги с 50 и более оценками) - J.K. Rowling/Mary GrandPré, среднее средних оценок его книг (с 50 и более оценками) - 4.28;
- Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33.

![Апплодисменты](https://i.gifer.com/DhG5.gif "Хлоп-хлоп!")

<div class="alert alert-success"><b>Комментарий ревьюера 2</b>&nbsp;👍<br>
🎉 Поздравляю с успешным завершением последнего проекта финального спринта! 🎂
</div>